# 2) Wiener Linien API Verspätungen + MongoDB

Dieses Notebook startet einen FastAPI-Proxy innerhalb des Jupyter-Containers.

Zuerst FETCH_RBLS in der docker-compose.yml auf „true“ setzen, um den automatischen Abruf (Auto-Fetch) zu aktivieren. Danach werden die Zellen ausgeführt, um die API auf Port 8000 zu starten und die Stop-Zelle verwendet, um die API zu beenden, ohne Jupyter zu stoppen. Nun gibt es einen Zugriff vom Host-System: http://localhost:8000/health

Folgende Daten schauen wir uns hier näher an: Verspätungsminuten, Zeiten, Linien, Störungsereignisse


In [1]:
import sys
from pathlib import Path

candidates = [
    Path("/home/jovyan/api"),
    Path.cwd().parent / "api",
    Path.cwd() / "api",
]

api_root = next((p for p in candidates if (p / "app" / "main.py").exists()), None)
if api_root is None:
    raise FileNotFoundError("Could not find api/app/main.py. Check your volume mount or repo path.")

sys.path.insert(0, str(api_root))

from app.main import app


In [2]:
import threading
import uvicorn

def _build_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    return uvicorn.Server(config)

if "_api_thread" not in globals() or not _api_thread.is_alive():
    _api_server = _build_server()
    _api_thread = threading.Thread(target=_api_server.run, daemon=True)
    _api_thread.start()

print("API running at http://localhost:8000")


API running at http://localhost:8000


INFO:     Started server process [238]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     172.18.0.1:47184 - "POST /historical/import HTTP/1.1" 200 OK


In [7]:
if "_api_server" in globals():
    _api_server.should_exit = True
    print("API stopping...")
else:
    print("API not running")


API stopping...


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [101]
